In [1]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
spark = (
    SparkSession
    .builder
    .appName("SparkSQLExampleApp")
    .master("local[4]")
    .getOrCreate()
)

spark.conf.set("spark.sql.shuffle.partition", 4)
#spark.conf.set("spark.sql.session.timeZone", "UTC")

22/06/17 12:17:24 WARN Utils: Your hostname, emif-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.171 instead (on interface en0)
22/06/17 12:17:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/17 12:17:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/17 12:17:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = (
    spark
    .read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Users/emif/Documents/spark-lab/datasets/mnm_dataset.csv")
)

In [4]:
df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



In [5]:
df.describe().show()

+-------+-----+------+-----------------+
|summary|State| Color|            Count|
+-------+-----+------+-----------------+
|  count|99999| 99999|            99999|
|   mean| null|  null|55.00090000900009|
| stddev| null|  null|26.26648202084822|
|    min|   AZ|  Blue|               10|
|    max|   WY|Yellow|              100|
+-------+-----+------+-----------------+



In [6]:
df.show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   WA|Yellow|   20|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
+-----+------+-----+
only showing top 20 rows



In [56]:
pivot_df = (
    df
    .groupBy("State")
    .pivot("Color")
    .avg("Count")
)

In [57]:
pivot_df.show()

+-----+------------------+------------------+------------------+------------------+------------------+------------------+
|State|              Blue|             Brown|             Green|            Orange|               Red|            Yellow|
+-----+------------------+------------------+------------------+------------------+------------------+------------------+
|   AZ| 54.99449877750611|54.350412249705535| 54.82219570405728| 54.28300769686205|54.637135922330096| 54.98548972188634|
|   OR| 54.99756986634265| 54.98827884022209|54.821297429620564| 54.22489959839358| 54.88510638297873| 54.60285006195787|
|   WY| 54.68870192307692| 56.20757180156658|55.657227138643066|55.144827586206894|54.950898203592814|53.997539975399754|
|   NV|53.797369994022716| 55.81050090525045| 53.78739693757362|54.865070093457945|  55.4944099378882|54.561194029850746|
|   CA| 55.59762944479102|55.740395809080326|54.268717353453276|54.502715751357876| 55.26992753623188|  55.8693967902601|
|   WA|55.31446153846153

In [55]:
def round_columns(df, columns, precision=2):
    for col in columns:
        df = df.withColumn(col, F.round(F.col(col), precision))
    return df

In [59]:
final_df = round_columns(pivot_df, ["Blue", "Brown", "Green", "Orange", "Red", "Yellow"])

In [60]:
final_df.show()

+-----+-----+-----+-----+------+-----+------+
|State| Blue|Brown|Green|Orange|  Red|Yellow|
+-----+-----+-----+-----+------+-----+------+
|   AZ|54.99|54.35|54.82| 54.28|54.64| 54.99|
|   OR| 55.0|54.99|54.82| 54.22|54.89|  54.6|
|   WY|54.69|56.21|55.66| 55.14|54.95|  54.0|
|   NV| 53.8|55.81|53.79| 54.87|55.49| 54.56|
|   CA| 55.6|55.74|54.27|  54.5|55.27| 55.87|
|   WA|55.31|55.77|54.24|  55.2|55.85| 55.87|
|   NM|55.04|55.39| 54.2| 54.81|56.03| 54.94|
|   TX|54.81|55.29|55.13| 55.88|55.31| 55.09|
|   CO|55.11|56.58|54.71|  55.4|55.09| 55.22|
|   UT|54.37|54.55|55.56| 54.24|54.16| 54.26|
+-----+-----+-----+-----+------+-----+------+

